# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [2]:
def score(param, val):
    with open('apache.json') as f:
        rules = json.load(f)
    range_list = rules[param]
    if val<0:
        val = 0
    elif val>998:
        val = 998
    points = 0
    for r in range_list:
        if val>=r["min"] and val < r["max"]:
            points = r["points"]
            break
    return points

def creatine_score(val, chronic_failure = False, accute_failure = False):
    with open('apache.json') as f:
        rules = json.load(f)
    if accute_failure:
        range_list = rules["Creatinine"]["Acute Renal Failure"]
    elif chronic_failure:
        range_list = rules["Creatinine"]["Chronic Renal Failure"]
    else:
        range_list = rules["Creatinine"]["None"]
    if val<0:
        val = 0
    elif val>998:
        val = 998
    points = 0
    for r in range_list:
        if val>=r["min"] and val < r["max"]:
            points = r["points"]
            break
    return points

def fio2_score(fio2, aa_gradient = 0, pao2 = 0):
    with open('apache.json') as f:
        rules = json.load(f)
    if fio2<50:
        range_list = rules["FiO2"]["PaO2"]
        val = pao2
    else:
        range_list = rules["FiO2"]["A-a gradient"]
        val = aa_gradient
    if val<0:
        val = 0
    elif val>998:
        val = 998
    points = 0
    for r in range_list:
        if val>=r["min"] and val < r["max"]:
            points = r["points"]
            break
    return points

def organ_score(failure_type):
    with open('apache.json') as f:
        rules = json.load(f)
    return rules["Organ Failure History"][failure_type]

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [3]:
assert(score("Age",15) == 0)
assert(score("Age",47) == 2)
assert(score("Age",80) == 6)

assert(score("Temperature",80) == 4)
assert(score("Temperature",38.5) == 1)
assert(score("Temperature",34) == 1)

assert(score("pH",7.7) == 4)
assert(score("pH",7.5) == 1)
assert(score("pH",7.33) == 0)

assert(score("Heart Rate",180) == 4)
assert(score("Heart Rate",140) == 3)
assert(score("Heart Rate",55) == 2)

assert(score("Respiratory Rate",50) == 4)
assert(score("Respiratory Rate",25) == 1)
assert(score("Respiratory Rate",12) == 0)

assert(score("Sodium",180) == 4)
assert(score("Sodium",150) == 1)
assert(score("Sodium",120) == 2)

assert(score("Potassium",6) == 3)
assert(score("Potassium",3.5) == 0)
assert(score("Potassium",2.5) == 2)

assert(score("Hematocrit",60) == 4)
assert(score("Hematocrit",50) == 2)
assert(score("Hematocrit",40) == 0)

assert(score("White Blood Count",40) == 4)
assert(score("White Blood Count",20) == 2)
assert(score("White Blood Count",15) == 1)

assert(creatine_score(0.6, chronic_failure=True) == 0)
assert(creatine_score(1.5,accute_failure=True) == 4)
assert(creatine_score(0.6) == 0)

assert(fio2_score(55,aa_gradient=350)==3 )
assert(fio2_score(45,pao2=80) == 0)
assert(fio2_score(70,500) == 4)

assert(organ_score("None") == 0)
assert(organ_score("Elective") == 2)
assert(organ_score("Emergency") ==5)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [4]:
metrics = ["Organ Failure History", "Age","Temperature","pH","Heart rate","Respiratory rate","Sodium","Potassium","Creatinine","Hematocrit","White Blood Count","FiO2"]
def apache_score(metric_dict):
    ret = 0
    for m in metric_dict:
        if m in {"Patient", "Acute", "Acute Renal Failure", "PaO2", "A-a Gradient"}:
            continue
        if m == "FiO2":
            ret += fio2_score(metric_dict[m], metric_dict["A-a Gradient"], metric_dict["PaO2"])
        elif m=="Organ Failure History":
            ret += organ_score(metric_dict[m])
        elif m=="Creatinine":
            v1, v2 = False, False
            if metric_dict["Acute Renal Failure"] == "Acute Renal Failure":
                v1 = True
            else:
                v2 = True
            ret += creatine_score(metric_dict[m], chronic_failure = v2, accute_failure = v1)
        elif m=="Glasgow Coma Scale":
            ret += 15 - metric_dict["Glasgow Coma Scale"]
        else:
            ret += score(m, metric_dict[m])
    return ret

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [5]:
import pandas as pd
patients = pd.read_csv("TestPatients.csv")
scores = pd.read_csv("Scores_corrected.csv")

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [8]:
inc = []
for i in range(len(patients)):
    pred = apache_score(dict(patients.loc[i]))
    true = int(scores.loc[i])
    try:
        assert(pred == true)
    except:
        inc.append(i)
        print(i, pred, true)